<div align='center'><font size="5" color='#353B47'>Introduction to Time Series Analysis</font></div>
<div align='center'><font size="4" color="#353B47">Good practices and forecasting methods</font></div>
<br>
<hr>

**<font color="blue" size="4">What is a Time Series ?</font>**

> A Time Series is a **time-indexed** series of data. In Finance, a time series tracks the movement of the chosen data points, such as a *security’s price*, over a specified period of time with data points recorded at **regular intervals**.

<img src="https://toocooltrafficschool.com/wp-content/uploads/2019/02/what-is-tlsae740.png" width="500">

**<font color="blue" size="4">Why is it used for ?</font>**

> Time series analysis can be useful to see how a given asset, security, or economic variable changes over time. It can also be used to examine how the changes associated with the chosen data point compare to shifts in other variables over the same time period.

> For example, suppose you wanted to analyze a time series of daily closing stock prices for a given stock over a period of one year. You would obtain a list of all the closing prices for the stock from each day for the past year and list them in chronological order. 

# Import libraries

In [ ]:
# Regular libraries for data manipulation
import pprint
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Visualization
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical tools for time series analysis
from scipy import signal
import statsmodels.api as sm
!pip install pmdarima
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.tools import diff
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller, kpss, acf, grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf,month_plot,quarter_plot
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

sns.set_style("whitegrid")
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15)

from pylab import rcParams

# Import data

**<font color="blue" size="4">What is the source of the data ?</font>**

The data is originally imported from a preprocessed dataset I created scrapping <a href="https://markets.businessinsider.com/stocks/air_liquide-stock">this website</a> via *beautifulsoup* library. It is composed of daily stocks of 40 most valuables French Companies (CAC40) from January 2010 to April 2020. 

**For this notebook, I will only focus on Air Liquide Stocks.**

In [ ]:
data = pd.read_csv('../input/cac40-stocks-dataset/preprocessed_CAC40.csv', usecols = ['Name','Date','Open','Closing_Price','Daily_High','Daily_Low','Volume'])

In [ ]:
# Filter the dataframe on Air Liquide stocks
air_liquide = data[data['Name'] == 'Air Liquide'].copy()

In [ ]:
# Converting 'Date' to datetime object
air_liquide['Date'] = pd.to_datetime(air_liquide['Date'])
air_liquide['Year'] = air_liquide['Date'].dt.year
air_liquide['Month'] = air_liquide['Date'].dt.month

In [ ]:
print(f'air_liquide shape: {air_liquide.shape[0]} rows, {air_liquide.shape[1]} columns')

In [ ]:
air_liquide.head()

## <div id="summary">Summary</div>

**<font size="2"><a href="#chap1">1. Line plot of observations</a></font>**
**<br><font size="2"><a href="#chap2">2. Treatment of missing values</a></font>**
**<br><font size="2"><a href="#chap3">3. Seasonality</a></font>**
**<br><font size="2"><a href="#chap4">4. Time series components</a></font>**
**<br><font size="2"><a href="#chap5">5. Stationarity</a></font>**
**<br><font size="2"><a href="#chap6">6. Autocorrelation</a></font>**
**<br><font size="2"><a href="#chap7">7. Lag Scatter Plot</a></font>**
**<br><font size="2"><a href="#chap8">8. Moving Average</a></font>**
**<br><font size="2"><a href="#chap9">9. Exponential Smoothing</a></font>**
**<br><font size="2"><a href="#chap10">10. Forecasting with Auto ARIMA</a></font>**

# <div id="chap1">1. Line plot of observations</div>

A **line plot** is the a graph that displays data as points or check marks above a number line, showing the frequency of each value. It helps to identify:

* **Outliers** or exagerated values

* **Missing observations** (in most cases an error messages appear)

* **Variation** of the data

* **Seasonality and trend** of the data

In [ ]:
# Line plot
fig, ax = plt.subplots(figsize=(15, 6))
sns.lineplot(air_liquide['Date'], air_liquide['Open'] )

# Formatting
ax.set_title('Open Price', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax.set_xlabel('Year', fontsize = 16, fontdict=dict(weight='bold'))
ax.set_ylabel('Price', fontsize = 16, fontdict=dict(weight='bold'))
plt.tick_params(axis='y', which='major', labelsize=16)
plt.tick_params(axis='x', which='major', labelsize=16)

**<font color="blue" size="4">What can we say about Air Liquide stocks ?</font>**

The data **don't seem to show seasonality** within each year. There is rather a **trend** than a **cyclic behavior**. We can see that the price is falling sharply early 2020 due to the COVID19.

**<font size="2"><a href="#summary">Back to summary</a></font>**

----

# <div id="chap2">2. Treatment of missing values</div>

Most of the time, the problem must be dealt with on a case-by-case basis. But there are different ways to input missing values:

* **Forward filling and Backward Fill**: Forward filling means fill missing values with previous data. Backward filling means fill missing values with next data point.

* **Linear interpolation**: Method of curve fitting using linear polynomials to construct new data points within the range of a discrete set of known data points.

* **Quadratic interpolation**: Method of curve fitting using non linear polynomials to construct new data points within the range of a discrete set of known data points.

* **Mean of nearest neighbors**

* **Mean of seasonal**

In [ ]:
# Let's check NaN 
air_liquide.isna().sum()

1 missing value in <font color='red'>'Open'</font> column and 134 misising values in <font color='red'>'Volume'</font> column

In [ ]:
# Get index of the missing value in Open column
index_open_missing = air_liquide[pd.isnull(air_liquide['Open'])].index
print(list(index_open_missing)[0])

In [ ]:
# I replace the missing value by the Open price value of the previous day (backward fill)
air_liquide['Open'] = air_liquide['Open'].fillna(method='bfill')

In [ ]:
print(f"Percentage of missing values in Volume: {round(sum(pd.isnull(air_liquide['Volume']))/air_liquide.shape[0],2)}\n")
print(air_liquide[air_liquide['Volume']==0])

Missing values of the <font color='red'>'Volume'</font> column correspond to 5% of observations. Plus, There is no observation for which the value is 0. Let's make the assumption that a missing value corresponds to 0 volume of stocks traded.

In [ ]:
# Imputation
imputer = SimpleImputer(strategy='constant', fill_value=0)
air_liquide_plus = imputer.fit_transform(air_liquide)

# Imputation removed column names; put them back
imputed_air_liquide = pd.DataFrame(air_liquide_plus)
imputed_air_liquide.columns = air_liquide.columns

In [ ]:
# Replace , by . so Volume can be converted as float
imputed_air_liquide['Volume'] = imputed_air_liquide['Volume'].apply(lambda x : str(x))
imputed_air_liquide['Volume'] = pd.to_numeric(imputed_air_liquide['Volume'].apply(lambda x : x.replace(',','',1)))

# Convert object to numeric 
imputed_air_liquide['Open'] = pd.to_numeric(imputed_air_liquide['Open'])
imputed_air_liquide['Closing_Price'] = pd.to_numeric(imputed_air_liquide['Closing_Price'])
imputed_air_liquide['Daily_High'] = pd.to_numeric(imputed_air_liquide['Daily_High'])
imputed_air_liquide['Daily_Low'] = pd.to_numeric(imputed_air_liquide['Daily_Low'])

In [ ]:
# Plot Daily Volume Lineplot
fig, ax = plt.subplots(figsize=(15, 6))
sns.lineplot(imputed_air_liquide['Date'], imputed_air_liquide['Volume'] )

ax.set_title('Daily Volume', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax.set_xlabel('Year', fontsize = 16, fontdict=dict(weight='bold'))
ax.set_ylabel('Price', fontsize = 16, fontdict=dict(weight='bold'))
plt.tick_params(axis='y', which='major', labelsize=16)
plt.tick_params(axis='x', which='major', labelsize=16)

There is a lot of noise because I plotted on a daily scale. By chosing the mean of volume traded per month instead of considering the daily volume traded, the graphis likely to be more readable.

In [ ]:
# Aggregating the Time Series to a monthly scaled index
y = imputed_air_liquide[['Date','Volume']].copy()
y.set_index('Date', inplace=True)
y.index = pd.to_datetime(y.index)
y = y.resample('1M').mean()
y['Date'] = y.index

# Plot the Monthly Volume Lineplot
fig, ax = plt.subplots(figsize=(15, 6))
sns.lineplot(y['Date'], y['Volume'] )

ax.set_title('Monthly Volume', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax.set_xlabel('Year', fontsize = 16, fontdict=dict(weight='bold'))
plt.tick_params(axis='y', which='major', labelsize=16)
plt.tick_params(axis='x', which='major', labelsize=16)

The graph is still noisy but more readable. It's up to you to chose the best period interval and balance the readability/information compromise. 

* **Small period interval**: <font color='red'>Noisy signal</font>, <font color='green'>more information</font>
* **Big period interval**: <font color='green'>Easy to read signal</font>, <font color='red'>loss of information</font>

**<font size="2"><a href="#summary">Back to summary</a></font>**

----

# <div id="chap3">3. Seasonality</div>

**<font color="blue" size="4">What is seasonality ?</font>**

In Time Series data, seasonality is the presence of variations that **occur** at **specific regular intervals less than a year**, such as **weekly**, **monthly**, or **quarterly**.

In [ ]:
imputed_air_liquide['Year'] = imputed_air_liquide['Date'].dt.year
imputed_air_liquide['Month'] = imputed_air_liquide['Date'].dt.month

In [ ]:
imputed_air_liquide['Year'].unique()

In [ ]:
variable = 'Open'
fig, ax = plt.subplots(figsize=(15, 6))

sns.lineplot(imputed_air_liquide['Month'], imputed_air_liquide[variable], hue = imputed_air_liquide['Year'])
ax.set_title('Seasonal plot of Price', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax.set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax.set_ylabel('Price', fontsize = 16, fontdict=dict(weight='bold'))
ax.legend(labels = [str(2010+i) for i in range(11)], bbox_to_anchor=(1.1, 1.05))

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

sns.boxplot(imputed_air_liquide['Year'], imputed_air_liquide[variable], ax=ax[0])
ax[0].set_title('Year-wise Box Plot\n(The Trend)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[0].set_xlabel('Year', fontsize = 16, fontdict=dict(weight='bold'))
ax[0].set_ylabel('Price', fontsize = 16, fontdict=dict(weight='bold'))

sns.boxplot(imputed_air_liquide['Month'], imputed_air_liquide[variable], ax=ax[1])
ax[1].set_title('Month-wise Box Plot\n(The Seasonality)', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax[1].set_xlabel('Month', fontsize = 16, fontdict=dict(weight='bold'))
ax[1].set_ylabel('Price', fontsize = 16, fontdict=dict(weight='bold'))

fig.autofmt_xdate()

**Seasonal plot of Price**

We can see directly that 2019 was a very good year for the firm. The share price rose from 95 euros to 125 euros. This is the biggest increase over the last 10 years.

**Year wise Box Plot**

Few outliers, no seasonality and an uptrend for the last decade except for 2016. This small slowdown is characterized by the completion of the Airgas acquisition, in 2016, and its first contribution to the Group's performance. You can read the full 2016 report <a href = 'https://www.airliquide.com/sites/airliquide.com/files/2016/07/29/air-liquide-h1-2016-results.pdf'>here</a>

**Month wise Box Plot**

The variance between each class is really small, it shows clearly that there is no effect of seasonality.

<img src = "https://static.vecteezy.com/system/resources/previews/000/126/617/non_2x/vector-tool-box.jpg">

<font color='blue' size='4'>A powerful visualization</font>

<br>On a whole, these plots allow you to:

* Check **seasonal pattern** if it exists
* Identity the **years in which the pattern changes**
* Identify **large jumps or drops**

In the trend and seasonality boxplot:

* Check instantaneously **trend and seasonality**
* Check **outliers**

<font color='blue' size='4'>A powerful visualization you can add to your toolbox</font>

In [ ]:
# Aggregating the Time Series to a monthly scaled index
y = imputed_air_liquide[['Date','Open','Closing_Price']].copy()
y.set_index('Date', inplace=True)
y.index = pd.to_datetime(y.index)
y = y.resample('1M').mean()

# The magic
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(16, 10))

month_plot(y['Open'], ax=ax[0]);
ax[0].set_ylabel('Open', fontsize = 16, fontdict=dict(weight='bold'))

month_plot(y['Closing_Price'], ax=ax[1]);
ax[1].set_ylabel('Closing_Price', fontsize = 16, fontdict=dict(weight='bold'))

This graph performs a groupby function to see more clearly the months of seasonality. We can see that there is no seasonality here. The mean of 10 years air liquide open and closing price per month is quite the same.

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

# <div id="chap4">4. Time series components</div>

If we assume an additive decomposition, then we can write

$$y_t = S_t + T_t + R_t$$

where $y_t$ is the data, $S_t$ is the seasonal component, $T_t$ is the trend-cycle component and $R_t$ is the residual component, all at period $t$.
Also,for a multiplicative decomposition, we have

$$y_t = S_t * T_t * R_t$$

The additive decomposition is the most appropriate if the magnitude of the seasonal fluctuations, or the variation around the trend-cycle, does not vary with the level of the time series. When the variation in the seasonal pattern, or the variation around the trend-cycle, appears to be proportional to the level of the time series, then a multiplicative decomposition is more appropriate. Multiplicative decompositions are common with economic time series.


In [ ]:
# Aggregating the Time Series to a monthly scaled index
y = imputed_air_liquide[['Date','Open']].copy()
y.set_index('Date', inplace=True)
y.index = pd.to_datetime(y.index)
y = y.resample('1M').mean()

# Setting rcparams
rcParams['figure.figsize'] = 15, 12
rcParams['axes.labelsize'] = 20
rcParams['ytick.labelsize'] = 16
rcParams['xtick.labelsize'] = 16

# Using statistical tools of statsmodel library
decomposition = sm.tsa.seasonal_decompose(y, model='multiplicative', freq = 12)
decomp = decomposition.plot()
decomp.suptitle('Open decomposition', fontsize=22)

<font color='blue' size='4'>How to read this ?</font>

<br>The three components are shown separately in the bottom three panels. These components can be multiplied together to reconstruct the data shown in the top panel. We can see that the seasonal component changes slowly over time. But this doesn't mean years far apart won't have different seasonal patterns.

The residual component shown in the bottom panel is what is left over when the seasonal and trend-cycle components have been subtracted from the data.

If the seasonal component is removed from the original data, the resulting values are the “seasonally adjusted” data. For an additive decomposition, the seasonally adjusted data are given by $y_t - S_t$, and for multiplicative data, the seasonally adjusted values are obtained using $y_t / S_t$.

**<font size="2"><a href="#summary">Back to summary</a></font>**

----

# <div id="chap5">5. Stationarity</div>

<font color='blue' size='4'>What describes best stationarity ?</font>

<br>A stationary Time Series is one whose properties do not depend on the time at which the series is observed. Thus, time series with trends, or with seasonality, are not stationary 

A time series with cyclic behavior (but with no trend or seasonality) is stationary.

<font color='blue' size='4'>What is stationarity used for ?</font>

<br>Most statistical forecasting methods are designed to work on a stationary time series. The first step in the forecasting process is typically to do some transformation to convert a non-stationary series to stationary. Forecasting a stationary series is relatively easier and the forecasts are more reliable.

We know that linear regression works best if the predictors (X variables) are not correlated against each other. So, stationarizing the series solves this problem since it removes any persistent autocorrelation, thereby making the predictors (lags of the series) in the forecasting models nearly independent.

<font color='blue' size='4'>Really interesting, but how to make a Time Series stationary ?</font>

<br>Well, several ways exist:

* Difference the series once or more times (subtracting the next value by the current value)

* Take the log of the series (helps to stabilize the variance of a time series.)

* Take the $n_{th}$ root of the series

* Combinations of the above

But first, to test if a time series is stationary we can:

* Look at the time plot.

* Split the series into 2 parts and compute descriptive statistics. If they differ, then it is not stationary.

* Perform statistical tests called Unit Root Tests like Augmented Dickey Fuller test (ADF Test), Kwiatkowski-Phillips-Schmidt-Shin — KPSS test (trend stationary), and Philips Perron test (PP Test).

The most commonly used is the ADF test, where the null hypothesis is that the time series possesses a unit root (or random walk with drift) and is non-stationary. So, if the P-Value in ADF test is less than the significance level (0.05), you reject the null hypothesis and the series is stationary.

In [ ]:
# check for stationarity
def adf_test(series, title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print('Augmented Dickey-Fuller Test: {}'.format(title))
    # .dropna() handles differenced data
    result = adfuller(series.dropna(),autolag='AIC') 
    
    labels = ['ADF test statistic','p-value','# lags used','# observations']
    out = pd.Series(result[0:4],index=labels)

    for key,val in result[4].items():
        out['critical value ({})'.format(key)]=val
        
    # .to_string() removes the line "dtype: float64"
    print(out.to_string())          
    
    if result[1] <= 0.05:
        print("Strong evidence against the null hypothesis")
        print("Reject the null hypothesis")
        print("Data has no unit root and is stationary")
    else:
        print("Weak evidence against the null hypothesis")
        print("Fail to reject the null hypothesis")
        print("Data has a unit root and is non-stationary")
        
# Aggregating the Time Series to a monthly scaled index
y = imputed_air_liquide[['Date','Open']].copy()
y.set_index('Date', inplace=True)
y.index = pd.to_datetime(y.index)
y = y.resample('1M').mean()
        
adf_test(y['Open'],title='') 

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2,figsize=(15, 11))

y['OpenDiff1'] = diff(y['Open'],k_diff=1)
y['OpenDiff2'] = diff(y['Open'],k_diff=2)
y['OpenDiff3'] = diff(y['Open'],k_diff=3)

y['Open'].plot(title="Initial Data",ax=ax[0][0]).autoscale(axis='x',tight=True);
y['OpenDiff1'].plot(title="First Difference Data",ax=ax[0][1]).autoscale(axis='x',tight=True);
y['OpenDiff2'].plot(title="Second Difference Data",ax=ax[1][0]).autoscale(axis='x',tight=True);
y['OpenDiff3'].plot(title="Third Difference Data",ax=ax[1][1]).autoscale(axis='x',tight=True);

fig.autofmt_xdate()

**<font size="2"><a href="#summary">Back to summary</a></font>**

----

# <div id="chap6">6. Autocorrelation</div>

<font color='blue' size='4'>Correlation, Autocorrelation, Partial Autocorrelation ? It looks like all the same, what is the difference ?</font>

<br>**Correlation** measures the extent of a linear relationship between two variables.

**Autocorrelation** measures the linear relationship between lagged values of a time series, for example between $y_t$ and $y_{t-1}$. If a series is significantly autocorrelated, that means, the previous values of the series (lags) may be helpful in predicting the current value.

![](http://)**Partial autocorrelations** measure the linear dependence of one variable after removing the effect of other variable(s) that affect both variables. That is, the partial autocorrelation at lag **k** is the autocorrelation between $y_t$ and $y_{t} + y_{t+k}$ that is not accounted for by lags 1 through k−1. 

We essentially plot out the relationship between the previous day’s/month’s residuals versus the real values of the current day. In general, we expect the partial autocorrelation to drop off quite quickly.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2,figsize=(15, 6))
autocorr = acf(imputed_air_liquide['Open'], nlags=60, fft=False)
print(autocorr)

plot_acf(imputed_air_liquide['Open'].tolist(), lags=60, ax=ax[0], fft=False);
plot_pacf(imputed_air_liquide['Open'].tolist(), lags=60, ax=ax[1]);

For autocorrelation, the y-axis is the value for the correlation between a value and its lag. The lag is on the x-axis. The zero-lag has a correlation of 1 because it correlates with itself perfectly. 

The autocorrelation plot shows that most of the spikes are not statistically significant. This indicates that the returns are not highly correlated, as shown here.

**<font size="2"><a href="#summary">Back to summary</a></font>**

-----

# <div id="chap7">7. Lag Scatter Plot</div>

<font color='blue' size='4'>Definition</font>

<br>A useful type of plot to explore the relationship between each observation and a lag of that observation is called the scatter plot.
It plots the observation at time t on the x-axis and the lag1 observation (t-1) on the y-axis.

* If the points cluster along a diagonal line from the bottom-left to the top-right of the plot, it suggests a **positive correlation** relationship.

* If the points cluster along a diagonal line from the top-left to the bottom-right, it suggests a **negative correlation** relationship.

* Either relationship is good as they can be modeled.

* More points tighter into the diagonal line suggests a stronger relationship and more spread from the line suggests a weaker relationship.

* A ball in the middle or a spread across the plot suggests a weak or no relationship.

In [ ]:
lag_plot(y['Open']);

Here we have a **positive correlation** relationship.

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

# <div id="chap8">8. Moving Average (MA)</div>

<font color='blue' size='4'>The maths behind MA</font>

<br>Moving average is the estimation of the trend-cycle at time t, and is obtained by averaging the values of the time series within k periods of t. Observations that are nearby in time are also likely to be close in value. Therefore, the average eliminates some of the randomness in the data, leaving a smooth trend-cycle component.

It can be written as

$$ \hat{T_t} = \frac{1}{m}\sum_{j=-k}^{k}y_{t+j}$$

where $m = 2k + 1$. That is, the estimate of the trend-cycle at time $t$ is obtained by averaging values of the time series within k periods of $t$.

* Smaller windows will lead to more noise.

* It will always lag by the size of the window.

* It will never reach the full peak or valley of the data due to the averaging.

* It does not really inform you about possible future behavior, all it really does is describe trends in your data.

* Extreme historical values can skew MA significantly.

In [ ]:
# Aggregating the Time Series to a monthly scaled index
y = imputed_air_liquide[['Date','Open']].copy()
y.set_index('Date', inplace=True)
y.index = pd.to_datetime(y.index)
y = y.resample('1M').mean()

y['MA3'] = y.rolling(window=3).mean() 
y.plot(figsize=(15,6));

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

# <div id="chap9">9. Exponential Smoothing</div>

Exponential smoothing assigns **exponentially decreasing weights** for **newest** to **oldest** observations. In other words, the older the data, the less priority (“weight”) the data is given. Newer data is seen as more relevant and is assigned more weight. Smoothing parameters usually denoted by $\alpha$ ( $0<\alpha\leq1$) determine the weights for observations. Exponential smoothing is usually used to make **short term forecasts**, as longer term forecasts using this technique **can be quite unreliable**.

In [ ]:
# Setting parameters value
span = 3
# Weights of observations
alpha = 2/(span+1)

# Plot Simple exponential smoothing
y['ES3'] = SimpleExpSmoothing(y['Open']).fit(smoothing_level = alpha, optimized = False).fittedvalues.shift(-1)
y[['Open','ES3']].plot(figsize=(15,6));

<font color='blue' size='4'>Simple, Double, Triple exponential Smoothing ?</font>

<br>**Simple Exponential Smoothing** employs a **weighted moving average with exponentially decreasing weights**.

**Double Exponential Smoothing** adds a second smoothing factor $\beta$ ($0<\beta\leq1$) that addresses trends in the data. The benefit here is that the model can anticipate future increases or decreases where the level model would only work from recent calculations. We can also address different types of change (growth/decay) in the trend. If a time series displays a straight-line sloped trend, you would use an additive adjustment. If the time series displays an exponential (curved) trend, you would use a multiplicative adjustment.

**Triple Exponential Smoothing** (also called the Multiplicative Holt-Winters), the method is usually more reliable for parabolic trends or data that shows trends and seasonality.

In [ ]:
# Plot Double and Triple exponential smoothing
y['DESmul3'] = ExponentialSmoothing(y['Open'], trend = 'add').fit().fittedvalues.shift(-1)
y['TESmul3'] = ExponentialSmoothing(y['Open'], trend = 'add', seasonal = 'add', seasonal_periods = 12).fit().fittedvalues.shift(-1)
y[['Open', 'TESmul3', 'DESmul3']].plot(figsize = (15,6));

**<font size="2"><a href="#summary">Back to summary</a></font>**

----

# <div id="chap10">10. Forecasting with Auto ARIMA</div>

Now we are going to create an ARIMA model and will train it with the closing price of the stock on the train data. So let us split the data into training and test set and visualize it.

In [ ]:
# Reverse index so the dataframe is from oldest to newest values
imputed_air_liquide = imputed_air_liquide.reindex(index=imputed_air_liquide.index[::-1])

In [ ]:
# Split data into train and validation set 90/10
air_liquide_train, air_liquide_val = imputed_air_liquide[:int(len(imputed_air_liquide)*0.9)], imputed_air_liquide[int(len(imputed_air_liquide)*0.9):]

# Index disappeared, put them back
air_liquide_val = air_liquide_val.set_index('Date', drop=False)
air_liquide_train = air_liquide_train.set_index('Date', drop=False)

# Line plot
fig, ax = plt.subplots(figsize=(15, 6))
sns.lineplot(air_liquide_train['Date'], air_liquide_train['Open'], color = 'black')
sns.lineplot(air_liquide_val['Date'], air_liquide_val['Open'], color = 'blue')

# Formatting
ax.set_title('Open Price', fontsize = 20, loc='center', fontdict=dict(weight='bold'))
ax.set_xlabel('Year', fontsize = 16, fontdict=dict(weight='bold'))
ax.set_ylabel('Price', fontsize = 16, fontdict=dict(weight='bold'))
plt.tick_params(axis='y', which='major', labelsize=16)
plt.tick_params(axis='x', which='major', labelsize=16)
plt.legend(loc='upper right' ,labels = ('train', 'test'))

Its time to choose parameters p,q,d for ARIMA model. Last time we chose the value of p,d, and q by observing the plots of ACF and PACF but now we are going to use Auto ARIMA to get the best parameters without even plotting ACF and PACF graphs.

Auto ARIMA: Automatically discover the optimal order for an ARIMA model.
The auto_arima function seeks to identify the most optimal parameters for an ARIMA model, and returns a fitted ARIMA model. This function is based on the commonly-used R function, forecast::auto.arima.
The auro_arima function works by conducting differencing tests (i.e., Kwiatkowski–Phillips–Schmidt–Shin, Augmented Dickey-Fuller or Phillips–Perron) to determine the order of differencing, d, and then fitting models within ranges of defined start_p, max_p, start_q, max_q ranges. If the seasonal optional is enabled, auto_arima also seeks to identify the optimal P and Q hyper- parameters after conducting the Canova-Hansen to determine the optimal order of seasonal differencing, D.

In [ ]:
%%time
model_autoARIMA = auto_arima(air_liquide_train['Open'])

""", 
                             start_p = 0, 
                             start_q = 0,
                             test = 'adf', # use adftest to find optimal 'd'
                             max_p = 3,
                             max_q = 3, # maximum p and q
                             m = 7, # frequency of series
                             seasonal = False,
                             start_P = 0, 
                             D = 0, 
                             trace = True,
                             error_action = 'ignore',  
                             stepwise = True
"""

print(model_autoARIMA.summary())

In comment, you can see the parameters is used according to the documentation and the Time Series I have. However I wasn't really satisfied by the prediction (The best model was a ARMA(1,1) whose were decreasing whereas the trend is increasing since last 10 years.
Tbh, I got some difficulties tuning up this model, I guess <a href='https://alkaline-ml.com/pmdarima/tips_and_tricks.html'>this</a> could help, If you have any suggestions of a right way to do it, don't hesitate to add a comment, I would really appreciate.

Next, create an ARIMA model with provided optimal parameters p, d and q.

In [ ]:
model = ARIMA(air_liquide_train['Open'], order = (1, 1, 2))
# disp=-1: no output
fitted = model.fit(disp = -1)
print(fitted.summary())

Now let's start forecast the stock prices on the test dataset keeping 95% confidence level.

In [ ]:
# Forecast 260 next observations 
fc, se, conf = fitted.forecast(260, alpha=0.05)  # 95% confidence
fc_series = pd.Series(fc, index=air_liquide_val.index)
lower_series = pd.Series(conf[:, 0], index=air_liquide_val.index)
upper_series = pd.Series(conf[:, 1], index=air_liquide_val.index)

plt.figure(figsize=(12,5), dpi=100)
plt.plot(air_liquide_train['Open'], label='training')
plt.plot(air_liquide_val['Open'], color = 'blue', label='Actual Stock Price')
plt.plot(fc_series, color = 'orange',label='Predicted Stock Price')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.10)
plt.title('Air Liquide Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Actual Stock Price')
plt.legend(loc='upper left', fontsize=8)
plt.show()

As you can see our model did quite handsomely. Let us also check the commonly used accuracy metrics to judge forecast results:

In [ ]:
import math

In [ ]:
# Report performance
mse = mean_squared_error(air_liquide_val['Open'], fc)
print('MSE: '+str(mse))
mae = mean_absolute_error(air_liquide_val['Open'], fc)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(air_liquide_val['Open'], fc))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(fc - air_liquide_val['Open'])/np.abs(air_liquide_val['Open']))
print('MAPE: '+str(mape))

Around 11% MAPE(Mean Absolute Percentage Error) implies the model is about 89% accurate in predicting the test set observations. End to end, the value is close. However, the model could not predict the higher than normal growth for the past year, but it couldn't predict the covid19 crisis which involved a decreased in Open prices.

**<font size="2"><a href="#summary">Back to summary</a></font>**

---

# References

* https://www.machinelearningplus.com/time-series/time-series-analysis-python/

* https://www.udemy.com/python-for-time-series-data-analysis/

* https://machinelearningmastery.com/time-series-data-visualization-with-python/

* https://www.statisticshowto.com/exponential-smoothing/

* https://otexts.com/fpp2/components.html

<hr>
<br>
<div align='justify'><font color="#353B47" size="4">Thank you for taking the time to read this notebook. I hope that I was able to answer your questions or your curiosity and that it was quite understandable. <u>any constructive comments are welcome</u>. They help me progress and motivate me to share better quality content. I am above all a passionate person who tries to advance my knowledge but also that of others. If you liked it, feel free to <u>upvote and share my work.</u> </font></div>
<br>
<div align='center'><font color="#353B47" size="3">Thank you and may passion guide you.</font></div>